In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install google-api-python-client
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os
os.chdir('/content/drive/My Drive')

# Imports

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


# Load Panel Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Min_Charecteristics_Tilburg_1963_ALLSAMPLE.csv")

# Normalize data And lag feutures

In [ ]:
df['Mom1m'] = df['excess_return']
df['Y_excess_return'] = df['excess_return']
# Remove duplicates while keeping the last occurrence
df = df.sort_values('date').drop_duplicates(subset=['date', 'PERMNO'], keep='last')




df = df.sort_values(by=['PERMNO', 'date'])
# Shift the excess return by one month within each firm

# List of column names to lag
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted", "excess_return",
    "BM", "CompEquIss", "OperProf",  "MaxRet", "IndMom", "DolVol" ,"Mom1m" , "Mom6m", "Mom12m"
]


# Loop through each column in the list and shift them forward by one period within each group
for column in factor_columns:
    df[column] = df.groupby('PERMNO')[column].shift(1)




# List of column names to standardize
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf",  "MaxRet", "IndMom", "DolVol" ,"Mom1m" , "Mom6m", "Mom12m"
]
# Compute the mean and standard deviation for each factor column grouped by 'date'
means = df.groupby('date')[factor_columns].transform('mean')
stds = df.groupby('date')[factor_columns].transform('std')

# Standardize the existing columns without creating new ones
for column in factor_columns:
    df[column] = (df[column] - means[column]) / stds[column]


df = df.dropna()

# OLS

In [ ]:

df['date'] = pd.to_datetime(df['date'])

# Initialize a DataFrame to store all results
all_results_df = pd.DataFrame()

# Factor columns specified
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth", "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf", "MaxRet", "IndMom", "DolVol", "Mom1m", "Mom6m", "Mom12m"
]


for test_year in range(1999, 2023):
    # Update year ranges for training, validation, and test
    train_years = (1969, 1986 + test_year - 1999)
    val_years = (1987 + test_year - 1999, 1998 + test_year - 1999)
    test_years = (test_year, test_year)

    # Use masks for splitting the data
    train_mask = df['date'].dt.year.isin(range(train_years[0], train_years[1] + 1))
    val_mask = df['date'].dt.year.isin(range(val_years[0], val_years[1] + 1))
    test_mask = df['date'].dt.year.isin(range(test_years[0], test_years[1] + 1))

    # Split the data
    train_df = df[train_mask]
    val_df = df[val_mask]
    test_df = df[test_mask]

    # Debugging output
    print(f"Testing year: {test_year}")
    print(f"Train range: {train_years}, Entries: {len(train_df)}")
    print(f"Val range: {val_years}, Entries: {len(val_df)}")
    print(f"Test range: {test_years}, Entries: {len(test_df)}")

    # Check if the test_df is empty
    if test_df.empty:
        print(f"No data available for testing in year {test_year}. Skipping this year.")
        continue

    # Combine training and validation sets
    combined_train_val_df = pd.concat([train_df, val_df])

    # Prepare data
    X_combined_train_val = combined_train_val_df[factor_columns]
    X_combined_train_val = sm.add_constant(X_combined_train_val)
    y_combined_train_val = combined_train_val_df['Y_excess_return']

    # Fit the OLS model
    retrained_model = sm.OLS(y_combined_train_val, X_combined_train_val).fit()

    # Prepare test data
    X_test = test_df[factor_columns]
    X_test = sm.add_constant(X_test)
    y_test = test_df['Y_excess_return']

    # Predict and evaluate
    y_pred_test = retrained_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)

    # Collect results for this iteration
    results_df = pd.DataFrame({
        'PERMNO': test_df['PERMNO'],
        'Date': test_df['date'],
        'True_Y_excess_return': y_test,
        'Predicted_Y_excess_return': y_pred_test,
        'Test_MSE': test_mse
    })

    # Append results of this year to the all_results_df
    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

# Save the aggregated results to a single CSV file
all_results_df.to_csv('Thesis Tilburg GNN/ResultsDF/OLS_all_years_SP.csv', index=False)

print("Analysis complete. Results saved.")


# RF

In [ ]:



df['date'] = pd.to_datetime(df['date'])

# Initialize a DataFrame to store all results
all_results_df = pd.DataFrame()

# Factor columns specified
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth", "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf", "MaxRet", "IndMom", "DolVol", "Mom1m", "Mom6m", "Mom12m"
]

# Configuration for the RandomForestRegressor
n_estimators = 500  # Fixed number of trees
max_depth_options = [1, 2, 3, 4, 5, 6]  # Range of tree depths
max_features_options = [3, 5, 10, 20]  # Features considered at each split

# Loop over each test year from 1999 to 2022
for test_year in range(1999, 2023):
    train_years = (1969, 1986 + test_year - 1999)
    val_years = (1987 + test_year - 1999, 1998 + test_year - 1999)
    test_years = (test_year, test_year)

    # Create masks for splitting the data based on the specified years.
    train_mask = df['date'].dt.year.between(train_years[0], train_years[1])
    val_mask = df['date'].dt.year.between(val_years[0], val_years[1])
    test_mask = df['date'].dt.year.between(test_years[0], test_years[1])

    # Split the data into training, validation, and test sets.
    train_df = df[train_mask]
    val_df = df[val_mask]
    test_df = df[test_mask]

    if train_df.empty or val_df.empty or test_df.empty:
        print(f"Skipping year {test_year}: Missing data in one of the splits.")
        continue

    # Prepare the training and validation data sets
    X_train = train_df[factor_columns]
    y_train = train_df['Y_excess_return']
    X_val = val_df[factor_columns]
    y_val = val_df['Y_excess_return']

    # Hyperparameter tuning and model evaluation
    best_val_score = float('inf')
    best_params = None

    for max_depth in max_depth_options:
        for max_features in max_features_options:
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                max_features=max_features,
                random_state=42
            )
            model.fit(X_train, y_train)
            y_pred_val = model.predict(X_val)
            val_score = mean_squared_error(y_val, y_pred_val)

            if val_score < best_val_score:
                best_val_score = val_score
                best_params = {
                    'max_depth': max_depth,
                    'max_features': max_features
                }

    # Retrain model with the best parameters on combined dataset
    combined_train_val_df = pd.concat([train_df, val_df])
    X_combined_train_val = combined_train_val_df[factor_columns]
    y_combined_train_val = combined_train_val_df['Y_excess_return']
    retrained_model = RandomForestRegressor(
        n_estimators=n_estimators,
        **best_params,
        random_state=42
    )
    retrained_model.fit(X_combined_train_val, y_combined_train_val)

    # Predict on the test set
    X_test = test_df[factor_columns]
    y_test = test_df['Y_excess_return']
    y_pred_test = retrained_model.predict(X_test)

    # Calculate the test MSE for evaluation
    test_mse = mean_squared_error(y_test, y_pred_test)

    # Collect results for this iteration
    results_df = pd.DataFrame({
        'PERMNO': test_df['PERMNO'],
        'Date': test_df['date'],
        'True_Y_excess_return': y_test,
        'Predicted_Y_excess_return': y_pred_test,
        'Test_MSE': test_mse
    })

    # Append results of this year to the all_results_df
    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

    # Print an update after processing each year
    print(f"Completed analysis for year {test_year}. Test MSE: {test_mse}")

# Save the aggregated results to a single CSV file
all_results_df.to_csv('Thesis Tilburg GNN/ResultsDF/RF_all_years_SP.csv', index=False)

print("Analysis complete. Results saved.")


# XGBOOST

In [ ]:
warnings.filterwarnings('ignore')  # Ignore all warnings


df['date'] = pd.to_datetime(df['date'])

# Initialize an DataFrame to store all results
all_results_df = pd.DataFrame()


factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth", "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf", "MaxRet", "IndMom", "DolVol", "Mom1m", "Mom6m", "Mom12m"
]

# Loop through each test year from 1999 to 2022
for test_year in range(1999, 2023):

    train_years = (1969, 1986 + test_year - 1999)
    val_years = (1987 + test_year - 1999, 1998 + test_year - 1999)
    test_years = (test_year, test_year)

    # Createe masks for the training, validation, and test splits
    train_mask = df['date'].dt.year.between(*train_years)
    val_mask = df['date'].dt.year.between(*val_years)
    test_mask = df['date'].dt.year.between(*test_years)

    # Split the DataFrame into training, validation, and test sets
    train_df, val_df, test_df = df[train_mask], df[val_mask], df[test_mask]

    # Skip the year if any of the splits are empty
    if train_df.empty or val_df.empty or test_df.empty:
        print(f"Skipping year {test_year}: Missing data in one of the splits.")
        continue

    # Separate features and target variable for training and validation sets
    X_train, y_train = train_df[factor_columns], train_df['Y_excess_return']
    X_val, y_val = val_df[factor_columns], val_df['Y_excess_return']

    best_val_score, best_params = float('inf'), None

    # Hyperparameter tuning for XGBoost model
    for learning_rate in [0.01, 0.1]:
        for max_depth in [1, 2]:
            # Initialize and train the model
            model = xgb.XGBRegressor(
                n_estimators=30, max_depth=max_depth, learning_rate=learning_rate,
                objective='reg:squarederror', tree_method='gpu_hist', verbosity=0, eval_metric='rmse'
            )
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)

            # Predict on validation set and calculate validation score
            y_pred_val = model.predict(X_val, iteration_range=(0, model.best_iteration + 1))
            val_score = mean_squared_error(y_val, y_pred_val)

            # Update best parameters if current validation score is better
            if val_score < best_val_score:
                best_val_score = val_score
                best_params = {
                    'n_estimators': model.best_iteration + 1,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate
                }

    print(f"Year: {test_year}, Best validation MSE: {best_val_score}, Params: {best_params}")

    # Combine training and validation sets for final training
    combined_train_val_df = pd.concat([train_df, val_df])
    X_combined_train_val = combined_train_val_df[factor_columns]
    y_combined_train_val = combined_train_val_df['Y_excess_return']

    # Retrain the model on the combined training and validation set with the best parameters
    retrained_model = xgb.XGBRegressor(
        **best_params, objective='reg:squarederror', tree_method='gpu_hist', verbosity=0
    )
    retrained_model.fit(X_combined_train_val, y_combined_train_val)

    # Predict on test set and calculate test MSE
    X_test = test_df[factor_columns]
    y_test = test_df['Y_excess_return']
    y_pred_test = retrained_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)

    # Create a DataFrame to store results for the current year
    results_df = pd.DataFrame({
        'PERMNO': test_df['PERMNO'],
        'Date': test_df['date'],
        'True_Y_excess_return': y_test,
        'Predicted_Y_excess_return': y_pred_test,
        'Test_MSE': test_mse
    })

    # Append current year results to the all results DataFrame
    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

# Save the results to a CSV file
all_results_df.to_csv('Thesis Tilburg GNN/ResultsDF/XGB_all_years.csv', index=False)
print("Analysis complete. Results saved.")

# NN3

In [ ]:



df['date'] = pd.to_datetime(df['date'])

# Initialize a DataFrame to store all results
all_results_df = pd.DataFrame()

# Factor columns specified
factor_columns = [
    "Beta", "RoE", "InvestPPEInv", "ShareIss5Y", "Accruals", "dNoa",
    "GP", "AssetGrowth",  "Investment", "market_cap_adjusted",
    "BM", "CompEquIss", "OperProf", "MaxRet", "IndMom", "DolVol", "Mom1m", "Mom6m", "Mom12m"
]

# Data preparation and model configuration
def build_model(l1_penalty, learning_rate, input_shape):
    model = keras.Sequential([
        layers.Dense(32, activation='relu', input_shape=(input_shape,), kernel_regularizer=regularizers.l1(l1_penalty)),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

for test_year in range(1999, 2023):
    # Update year ranges for training, validation, and test
    train_years = (1969, 1986 + test_year - 1999)
    val_years = (1987 + test_year - 1999, 1998 + test_year - 1999)
    test_years = (test_year, test_year)

    # Create masks for splitting the data basedd on the specified years.
    train_mask = df['date'].dt.year.isin(range(train_years[0], train_years[1] + 1))
    val_mask = df['date'].dt.year.isin(range(val_years[0], val_years[1] + 1))
    test_mask = df['date'].dt.year.isin(range(test_years[0], test_years[1] + 1))

    # Split the data into training, validation, and test sets.
    train_df = df[train_mask]
    val_df = df[val_mask]
    test_df = df[test_mask]

    # Prepare data
    X_train = train_df[factor_columns]
    y_train = train_df['Y_excess_return']
    X_val = val_df[factor_columns]
    y_val = val_df['Y_excess_return']

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Hyperparameter tuning setup
    l1_penalties = [1e-5, 1e-3]
    learning_rates = [0.001, 0.01]
    batch_size = 512
    epochs = 200

    best_val_mse = float('inf')
    best_config = None
    best_epochs = 0

    for l1_penalty in l1_penalties:
        for learning_rate in learning_rates:
            model = build_model(l1_penalty, learning_rate, X_train_scaled.shape[1])
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                                epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

            val_mse = min(history.history['val_loss'])
            optimal_epochs = early_stopping.stopped_epoch - 5
            if val_mse < best_val_mse:
                best_val_mse = val_mse
                best_config = {'l1_penalty': l1_penalty, 'learning_rate': learning_rate}
                best_epochs = optimal_epochs if optimal_epochs > 0 else max(history.epoch)

    # Train 10 models and make predictions
    combined_train_val_df = pd.concat([train_df, val_df])
    X_combined_train_val = combined_train_val_df[factor_columns]
    y_combined_train_val = combined_train_val_df['Y_excess_return']
    X_combined_train_val_scaled = scaler.fit_transform(X_combined_train_val)
    X_test_scaled = scaler.transform(test_df[factor_columns])
    y_test = test_df['Y_excess_return']

    predictions = []
    for i in range(10):
        model = build_model(best_config['l1_penalty'], best_config['learning_rate'], X_combined_train_val_scaled.shape[1])
        model.fit(X_combined_train_val_scaled, y_combined_train_val, epochs=best_epochs, batch_size=512, verbose=0)
        y_pred = model.predict(X_test_scaled).flatten()
        predictions.append(y_pred)

    # Average predictions from all models
    y_pred_ensemble = np.mean(np.column_stack(predictions), axis=1)

    # Calculate the test MSE for evaluation
    test_mse = mean_squared_error(y_test, y_pred_ensemble)

    # Collect results for this iteration
    results_df = pd.DataFrame({
        'PERMNO': test_df['PERMNO'],
        'Date': test_df['date'],
        'True_Y_excess_return': y_test,
        'Predicted_Y_excess_return': y_pred_ensemble,
        'Test_MSE': test_mse
    })

    # Append results of this year to the all_results_df
    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

# Save the aggregated results to a single CSV file
all_results_df.to_csv('Thesis Tilburg GNN/ResultsDF/NN_all_years.csv', index=False)

print("Analysis complete. Results saved.")
